In [ ]:
# Import required libraries
from matplotlib import pyplot as plt
import numpy as np
import scipy

In [ ]:
class SoftEM:
    
    def __init__(self, K, tau_max=100):
        self.K = K               
        self.tau_max = tau_max  

        self.cluster_probs = None     # placeholder for mixing ratio
        self.word_probs = None      # placeholder for cluster member effective counts

    def fit(self, x):
        n_samples = x.shape[0]
        n_words = x.shape[1]
        # initialization:
        # start with equal probs for each cluster
        self.cluster_probs = np.array([1/self.K] * self.K) 
        # Start with random weightings for the words
        # Need to normalise so each row sums to 1
        # Numpy has some weird defaults, need to covert to 2D array
        self.word_probs = np.random.rand(self.K,n_words)
        word_probs_norm_coeff = np.sum(self.word_probs,axis=1,keepdims=True)
        self.word_probs = self.word_probs/np.array(word_probs_norm_coeff)
        
        terminate= False
        tau = 1
        # fitting loop - we iteratively take E and M steps until the termination criterion is met.
        cluster_probs_old = self.cluster_probs
        word_probs_old = self.word_probs
        cluster_preds = np.zeros((n_samples,self.K))
        while (not terminate):

            # E step:
            for n in range(0,n_samples):
                for k in range(0,self.K):
                    # calculate the log likelihood of cluster given doc based on the estimated cluster probs and word-cluster probs
                    cluster_raw_word_preds = np.multiply(x[n], np.where(self.word_probs[k]>0,np.log(self.word_probs[k]),-100))
                    cluster_word_preds_sum = np.sum(cluster_raw_word_preds)
                    cluster_preds[n,k] = np.log(self.cluster_probs[k]) + cluster_word_preds_sum
                # Normalise the total probs to 1 for each doc
                # Also takes probs out of log space
                cluster_preds[n] = np.exp(cluster_preds[n])
                cluster_preds_norm_coeff = np.sum(cluster_preds[n])
                cluster_preds[n] = cluster_preds[n]/cluster_preds_norm_coeff

            # M step
            # Cluster probs
            self.cluster_probs = np.sum(cluster_preds,axis=0)/n_samples
            # Word probs
            for k in range(0,self.K):
                cluster_word_freqs = np.sum((cluster_preds[:,k].reshape(n_samples,1)*x),axis=0)
                cluster_word_freqs_norm_coeff = np.sum(cluster_word_freqs)
                self.word_probs[k] = cluster_word_freqs/cluster_word_freqs_norm_coeff

            # increase iteration counter
            tau +=1
            # check termination condition
            terminate = tau == self.tau_max or (np.array_equal(cluster_probs_old, self.cluster_probs) and np.array_equal(word_probs_old, self.word_probs))
            cluster_probs_old = self.cluster_probs
            word_probs_old = self.word_probs
        print("Finished fitting at iteration", tau)
        
        
    # In a clustering-context, `predict` is equivalent to obtaining cluster assignments for new data
    def predict(self, x):
        n_samples = x.shape[0]
        n_words = x.shape[1]
        cluster_preds = np.zeros((n_samples, self.K))
        for n in range(0,n_samples):
            for k in range(0,self.K):
                # calculate the log likelihood of cluster given doc based on the estimated cluster probs and word-cluster probs
                cluster_raw_word_preds = np.multiply(x[n], np.where(self.word_probs[k]>0,np.log(self.word_probs[k]),-100))
                cluster_word_preds_sum = np.sum(cluster_raw_word_preds)
                cluster_preds[n,k] = np.log(self.cluster_probs[k]) + cluster_word_preds_sum
            # Normalise the total probs to 1 for each doc
            # Also takes probs out of log space
            cluster_preds[n] = np.exp(cluster_preds[n])
            cluster_preds_norm_coeff = np.sum(cluster_preds[n])
            cluster_preds[n] = cluster_preds[n]/cluster_preds_norm_coeff
        return cluster_preds

In [ ]:
class HardEM:
    
    def __init__(self, K, tau_max=100):
        self.K = K               
        self.tau_max = tau_max  

        self.cluster_probs = None     # placeholder for mixing ratio
        self.word_probs = None      # placeholder for cluster member effective counts

    def fit(self, x):
        n_samples = x.shape[0]
        n_words = x.shape[1]
        # initialization:
        # start with equal probs for each cluster
        self.cluster_probs = np.array([1/self.K] * self.K) 
        # Start with random weightings for the words
        # Need to normalise so each row sums to 1
        # Numpy has some weird defaults, need to covert to 2D array
        self.word_probs = np.random.rand(self.K,n_words)
        word_probs_norm_coeff = np.sum(self.word_probs,axis=1,keepdims=True)
        self.word_probs = self.word_probs/np.array(word_probs_norm_coeff)
        
        terminate= False
        tau = 1
        # fitting loop - we iteratively take E and M steps until the termination criterion is met.
        cluster_probs_old = self.cluster_probs
        word_probs_old = self.word_probs
        cluster_preds = np.zeros((n_samples,self.K))
        while (not terminate):
            # E step:
            for n in range(0,n_samples):
                for k in range(0,self.K):
                    # calculate the log likelihood of cluster given doc based on the estimated cluster probs and word-cluster probs
                    cluster_raw_word_preds = np.multiply(x[n], np.where(self.word_probs[k]>0,np.log(self.word_probs[k]),-10))
                    cluster_word_preds_sum = np.sum(cluster_raw_word_preds)
                    cluster_preds[n,k] = np.log(self.cluster_probs[k]) + cluster_word_preds_sum
                # Make argmax of the preds 1, all the rest 0
                cluster_preds[n] = np.exp(cluster_preds[n])
                best_k = np.argmax(cluster_preds[n])
                cluster_preds[n] = np.zeros(self.K)
                cluster_preds[n,best_k] = 1

            # M step
            # Cluster probs
            self.cluster_probs = np.sum(cluster_preds,axis=0)/n_samples
            # Word probs
            for k in range(0,self.K):
                cluster_word_freqs = np.sum((cluster_preds[:,k].reshape(n_samples,1)*x),axis=0)
                cluster_word_freqs_norm_coeff = np.sum(cluster_word_freqs)
                self.word_probs[k] = cluster_word_freqs/cluster_word_freqs_norm_coeff


            # increase iteration counter
            tau +=1
            # check termination condition
            terminate = tau == self.tau_max or (np.array_equal(cluster_probs_old, self.cluster_probs) and np.array_equal(word_probs_old, self.word_probs))
            cluster_probs_old = self.cluster_probs
            word_probs_old = self.word_probs
        print("Finished fitting at iteration", tau)
        
        
    # In a clustering-context, `predict` is equivalent to obtaining cluster assignments for new data
    def predict(self, x):
        n_samples = x.shape[0]
        n_words = x.shape[1]
        cluster_preds = np.zeros((n_samples, self.K))
        for n in range(0,n_samples):
            for k in range(0,self.K):
                # calculate the log likelihood of cluster given doc based on the estimated cluster probs and word-cluster probs
                cluster_raw_word_preds = np.multiply(x[n], np.where(self.word_probs[k]>0,np.log(self.word_probs[k]),-10))
                cluster_word_preds_sum = np.sum(cluster_raw_word_preds)
                cluster_preds[n,k] = np.log(self.cluster_probs[k]) + cluster_word_preds_sum
            # Make argmax of the preds 1, all the rest 0
            cluster_preds[n] = np.exp(cluster_preds[n])
            best_k = np.argmax(cluster_preds[n])
            cluster_preds[n] = np.zeros(self.K)
            cluster_preds[n,best_k] = 1
        return cluster_preds